In [14]:
import time
from dcicutils import s3Utils
from dcicutils import ff_utils
from functions.wfr import *
from functions.wfr_settings import *
from functions.notebook_functions import *

# get key from keypairs.json
my_env = 'data'
my_auth = get_key('koray_data')

my_s3_util = s3Utils(env=my_env)
raw_bucket = my_s3_util.raw_file_bucket
out_bucket = my_s3_util.outfile_bucket

file_url = '/search/?type=FileFastq&status=uploading&status=upload failed&limit=all'
#file_url = '/search/?type=FileReference&status=uploading&status=upload failed&limit=all'
#file_url = '/search/?type=FileProcessed&status=uploading&status=upload failed&limit=all'
file_url = "/search/?lab.display_title=Job+Dekker%2C+UMMS&status=uploading&status=upload+failed&type=FileFastq"
all_files = ff_utils.search_metadata(file_url, key=my_auth)
print(len(all_files))

1485


In [16]:
run_md5 = input("Do you wanna run md5 if missing? (md5/none)")

from contextlib import contextmanager
import sys, os

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout


no_md5 = []
no_s3_file = []
printn = 0
counter = 0
for a_file in all_files:

    counter += 1
    # check for deleted or weird cases
    try:
        if a_file['status'] == 'deleted':
            print("Deleted File", a_file)
            continue
    except:
        print(a_file)
        break
        
    if counter-printn > 100:
        print(counter)
        print(len(no_md5), 'files had missing md5 calculation')
        print(len(no_s3_file), 'files did not have s3 files')
        printn = counter

    if 'FileProcessed' in a_file['@type']:
            my_bucket = out_bucket
    else:  # covers cases of FileFastq, FileReference, FileMicroscopy
            my_bucket = raw_bucket       
 
    # check if file is in s3
    with suppress_stdout():
        head_info = my_s3_util.does_key_exist(a_file['upload_key'], my_bucket)
    
    if not head_info:
        no_s3_file.append(a_file['accession'])
        #print(a_file['accession'], a_file.get('aliases')[0], "does not have a file in S3")
        #print()
        continue
    print(head_info['LastModified'])
    continue
        
    file_id = a_file['accession']
    status = a_file.get('status')
    attributions = get_attribution(a_file)
    
    # is there md5sum for gzip
    if not a_file.get('md5sum'):
        print(file_id,"does not have the md5sum calculated during upload")
        no_md5.append(a_file['accession'])
    
    # Assumes workflow_runs come in time ordered list, and grabs the last ones for each wf run
    md5_report = get_wfr_out(a_file['uuid'], "md5", my_auth, md_qc = True, run=12)
    
    if md5_report['status'] == 'running':
        print(file_id, 'still running')
        continue 

    if status in ["uploading", "upload failed"] and md5_report['status'] != 'complete':
        # if not, shall we run it?
        if run_md5 == 'md5':
            print('md5 running for', file_id)
            inp_f = {'input_file':a_file['@id']}
            run_missing_wfr(step_settings('md5', 'no_organism', attributions), inp_f, file_id, my_auth, my_env, tag="")
            print()
        # user does not want it to be run, so just report
        else:
            print('md5 run missing for', file_id)

    elif status in ["uploading", "upload failed"] and md5_report['status'] == 'complete':
        print("ERROR:",file_id ,"There is a successful md5 run but status not switched")
        # if not, shall we run it?
        if run_md5 == 'md5':
            print('md5 running for', file_id)
            inp_f = {'input_file':a_file['@id']}
            run_missing_wfr(step_settings('md5', 'no_organism', attributions), inp_f, file_id, my_auth, my_env, tag="")
            print()
    else:
        print("SHOULD NOT PASS", a_file['accession'], md5_report)
        
#     elif md5_status != 'complete':
#         if a_file.get('content_md5sum'):
#             print('md5run not linked to file')
#             if run_md5 == 'md5':
#                 print('md5 running for', file_id)
#                 inp_f = {'input_file':a_file['@id']}
#                 run_missing_wfr(step_settings('md5', 'no_organism', attributions), inp_f, file_id, my_auth, my_env, tag="")
#                 print()
#         else:
#             print(file_id,md5_status, status, 'switched status file does not have md5 run or content_md5sum')
#             # if not, shall we run it?
#             if run_md5 == 'md5':
#                 print('md5 running for', file_id)
#                 inp_f = {'input_file':a_file['@id']}
#                 run_missing_wfr(step_settings('md5', 'no_organism', attributions), inp_f, file_id, my_auth, my_env, tag="")
#                 print()
#     print()

print(len(no_md5), 'files had missing md5 calculation')
print(len(no_s3_file), 'files did not have s3 files')


Do you wanna run md5 if missing? (md5/none)
2018-11-08 20:38:52+00:00
2018-11-08 20:38:46+00:00
2018-11-08 20:38:28+00:00
2018-11-08 20:38:23+00:00
2018-11-08 20:38:17+00:00
2018-11-08 20:37:59+00:00
2018-11-08 20:37:53+00:00
2018-11-08 20:37:47+00:00
2018-11-08 20:37:41+00:00
2018-11-08 20:37:35+00:00
2018-11-08 20:37:28+00:00
2018-11-08 20:37:21+00:00
2018-11-08 20:37:14+00:00
2018-11-08 20:37:08+00:00
2018-11-08 20:37:00+00:00
2018-11-08 20:36:55+00:00
2018-11-08 20:36:48+00:00
2018-11-08 20:36:43+00:00
2018-11-08 20:36:37+00:00
2018-11-08 20:36:29+00:00
2018-11-08 20:36:23+00:00
2018-11-08 20:36:17+00:00
2018-11-08 20:36:12+00:00
2018-11-08 20:34:40+00:00
2018-11-08 20:32:53+00:00
2018-11-08 20:30:12+00:00
2018-11-08 20:28:20+00:00
2018-11-08 20:26:22+00:00
2018-11-08 20:24:28+00:00
2018-11-08 20:22:23+00:00
2018-11-08 20:20:24+00:00
2018-11-08 20:18:27+00:00
2018-11-08 20:16:42+00:00
2018-11-08 20:14:50+00:00
2018-11-08 20:13:00+00:00
2018-11-08 20:11:11+00:00
2018-11-08 20:09:24+

2018-11-14 23:18:56+00:00
2018-11-14 23:18:35+00:00
2018-11-14 23:18:26+00:00
2018-11-14 23:18:20+00:00
2018-11-14 23:18:07+00:00
2018-11-14 23:17:46+00:00
2018-11-14 23:17:22+00:00
2018-11-14 23:16:43+00:00
2018-11-14 23:16:05+00:00
2018-11-14 23:15:56+00:00
2018-11-14 23:14:40+00:00
2018-11-14 23:14:14+00:00
2018-11-14 23:14:06+00:00
2018-11-14 23:13:26+00:00
2018-11-14 23:13:18+00:00
2018-11-14 23:12:01+00:00
2018-11-14 23:11:52+00:00
2018-11-14 23:11:47+00:00
2018-11-14 23:11:41+00:00
2018-11-14 23:30:20+00:00
2018-11-14 23:30:03+00:00
2018-11-14 23:29:25+00:00
2018-11-14 23:29:20+00:00
2018-11-14 23:29:14+00:00
2018-11-14 23:28:48+00:00
2018-11-14 23:28:35+00:00
2018-11-14 23:26:50+00:00
2018-11-14 23:26:43+00:00
2018-11-14 23:26:17+00:00
2018-11-14 23:25:20+00:00
2018-11-14 23:25:11+00:00
2018-11-14 23:24:45+00:00
2018-11-14 23:23:53+00:00
404
0 files had missing md5 calculation
61 files did not have s3 files
2018-11-14 23:23:47+00:00
2018-11-14 23:23:38+00:00
2018-11-14 23:23:30

2018-11-14 22:56:27+00:00
2018-11-14 22:55:31+00:00
2018-11-14 22:55:24+00:00
2018-11-14 22:55:16+00:00
2018-11-14 22:55:06+00:00
2018-11-14 22:53:50+00:00
2018-11-14 22:53:32+00:00
2018-11-14 22:53:21+00:00
2018-11-14 22:52:18+00:00
2018-11-14 22:51:53+00:00
2018-11-14 22:51:07+00:00
2018-11-14 22:51:01+00:00
2018-11-14 22:50:37+00:00
2018-11-14 22:50:24+00:00
2018-11-14 22:50:01+00:00
2018-11-14 22:49:31+00:00
2018-11-14 22:49:17+00:00
2018-11-14 22:49:09+00:00
2018-11-14 23:15:27+00:00
2018-11-14 23:15:17+00:00
2018-11-14 23:15:10+00:00
2018-11-14 23:14:47+00:00
2018-11-14 23:14:37+00:00
2018-11-14 23:14:00+00:00
2018-11-14 23:13:48+00:00
2018-11-14 23:13:39+00:00
2018-11-14 23:11:55+00:00
2018-11-14 23:11:39+00:00
707
0 files had missing md5 calculation
61 files did not have s3 files
2018-11-14 23:11:18+00:00
2018-11-14 23:11:10+00:00
2018-11-14 23:09:43+00:00
2018-11-14 23:09:27+00:00
2018-11-14 23:09:21+00:00
2018-11-14 23:09:03+00:00
2018-11-14 23:08:49+00:00
2018-11-14 23:08:26

2018-11-14 22:39:05+00:00
2018-11-14 22:38:58+00:00
2018-11-14 22:38:47+00:00
2018-11-14 22:38:18+00:00
2018-11-14 22:38:08+00:00
2018-11-14 22:37:57+00:00
2018-11-14 22:37:21+00:00
2018-11-14 22:37:06+00:00
2018-11-14 22:36:31+00:00
2018-11-14 22:36:24+00:00
2018-11-14 22:36:15+00:00
2018-11-14 22:36:07+00:00
2018-11-14 22:35:56+00:00
2018-11-14 22:35:47+00:00
2018-11-14 22:35:38+00:00
2018-11-14 22:35:01+00:00
2018-11-14 22:34:28+00:00
2018-11-14 22:33:34+00:00
2018-11-14 22:33:27+00:00
2018-11-14 22:33:17+00:00
1010
0 files had missing md5 calculation
61 files did not have s3 files
2018-11-14 22:33:10+00:00
2018-11-14 22:32:57+00:00
2018-11-14 22:32:25+00:00
2018-11-14 22:32:15+00:00
2018-11-14 22:32:08+00:00
2018-11-14 22:31:55+00:00
2018-11-14 22:31:43+00:00
2018-11-14 22:31:35+00:00
2018-11-14 22:31:11+00:00
2018-11-14 22:29:42+00:00
2018-11-14 22:45:16+00:00
2018-11-14 22:44:56+00:00
2018-11-14 22:44:21+00:00
2018-11-14 22:44:10+00:00
2018-11-14 22:43:47+00:00
2018-11-14 22:43:4

2018-11-14 22:36:24+00:00
2018-11-14 22:35:16+00:00
2018-11-14 22:29:35+00:00
2018-11-14 22:28:43+00:00
2018-11-14 22:24:04+00:00
2018-11-07 21:29:26+00:00
2018-11-07 21:29:17+00:00
2018-11-07 21:29:09+00:00
2018-11-07 21:29:01+00:00
2018-11-07 21:28:54+00:00
2018-11-07 21:28:46+00:00
2018-11-07 21:28:34+00:00
2018-11-07 21:28:24+00:00
1313
0 files had missing md5 calculation
61 files did not have s3 files
2018-11-07 21:28:14+00:00
2018-11-07 21:28:03+00:00
2018-11-07 21:27:53+00:00
2018-11-07 21:27:45+00:00
2018-11-07 21:27:40+00:00
2018-11-07 21:27:35+00:00
2018-11-07 21:27:26+00:00
2018-11-07 21:27:19+00:00
2018-11-07 21:27:10+00:00
2018-11-07 21:27:02+00:00
2018-11-07 21:26:51+00:00
2018-11-07 21:26:41+00:00
2018-11-07 21:26:32+00:00
2018-11-07 21:26:23+00:00
2018-11-07 21:26:10+00:00
2018-11-07 21:25:38+00:00
2018-11-07 21:25:31+00:00
2018-11-07 21:25:23+00:00
2018-11-14 23:11:07+00:00
2018-11-14 23:09:03+00:00
2018-11-14 23:08:33+00:00
2018-11-14 23:08:15+00:00
2018-11-14 23:07:0

In [ ]:
### Check if md5 runs are in sync with file status
import time
from dcicutils import s3Utils
from dcicutils import ff_utils
from functions.wfr import *
from functions.wfr_settings import *
from functions.notebook_functions import *

# get key from keypairs.json
my_env = 'data'
my_key = get_key('koray_data')


md5_runs = '/search/?run_status=complete&status=in+review+by+lab&type=WorkflowRunAwsem&workflow.title=md5'
md5_hits = ff_utils.search_metadata(md5_runs , key=my_key)

print(len(md5_hits))

prob = 0
for a_run in md5_hits:
    f_status = a_run['input_files'][0]['value']['status']
    if f_status != a_run['status']:
        print(f_status, a_run['status'])
        prob += 1
    
print(prob)


patch = True
need_patch = 0
need_patch_run = 0
counter = 0

In [ ]:

counter = 0
for a_run in md5_hits:
    counter += 1
    if counter % 100 == 0:
        print()
        print(counter)
    f_status = a_run['input_files'][0]['value']['status']
    if f_status in ['archived', 'released', 'released to project']:
        print(f_status, a_run['status'])
        ff_utils.patch_metadata({"status": f_status}, obj_id=a_run['uuid'] ,key=my_key)
print(counter)
    
       